# Sentiment Classification & Embedding II

* Embedding Layer
* Sequence Model

# 01. What data we use?

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data= pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_data= pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [3]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [5]:
train_data.shape, test_data.shape

((150000, 3), (50000, 3))

# (Skip) : 한국어 기본 전처리

In [ ]:
# !pip install konlpy

In [ ]:
# train_data=train_data.dropna(how='any')
# train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# # 한글과 공백을 제외하고 모두 제거
# train_data[:5]

In [ ]:
# import konlpy
# from konlpy.tag import Okt
# okt = Okt()
# stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# %%time

# X_train=[]
# for sentence in train_data['document']:
#     temp_X = []
#     temp_X=okt.morphs(sentence, stem=True) # 토큰화
#     temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
#     X_train.append(temp_X)

In [ ]:
# print(X_train[:4])

In [ ]:
# %%time

# test_data=test_data.dropna(how='any') # Null 값 제거
# test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행

# X_test=[]
# for sentence in test_data['document']:
#     temp_X = []
#     temp_X=okt.morphs(sentence, stem=True) # 토큰화
#     temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
#     X_test.append(temp_X)

# Tokenizing & Text to Sequences

In [6]:
## 위의 한국어 전처리를 했다면 사용하지 않는다.
## 명확히 str으로 데이터 타입을 변경해줘야 tokenizer에서 에러가 없다.
## 한국어 건들때만 그러는 것 같음.
X_train = train_data['document'].astype('str').tolist()
X_test = test_data['document'].astype('str').tolist()

In [11]:
X_train[:4]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정']

In [7]:
y_train = train_data['label'].values
y_test = test_data['label'].values

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_words = 35000   # 상위 35,000개의 단어만 보존
tokenizer = Tokenizer(num_words=max_words, lower=False)  # lower(대문자->소문자) 옵션은, 한국어를 할땐 끄자.

In [9]:
%%time
tokenizer.fit_on_texts( X_train )
x_train = tokenizer.texts_to_sequences(X_train)
x_test = tokenizer.texts_to_sequences(X_test)

CPU times: user 4.92 s, sys: 109 ms, total: 5.03 s
Wall time: 5.03 s


# Padding Sequence

In [ ]:
max_words = max_words ## 위에서 설정하길 35000
embedding_dim = 128 ## 단어 embedding 차원
max_len = 30 ## 문장 최대 길이

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
x_train = pad_sequences(x_train, maxlen = max_len)
x_test = pad_sequences(x_test, maxlen = max_len)

In [ ]:
x_train[:3]

In [ ]:
x_train[2]

In [ ]:
train_data.iloc[2]

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

# 0으로 가득찬 데이터에 대한 추가 전처리

In [ ]:
not0train_idx = x_train.sum(1) > 0
not0test_idx = x_test.sum(1) > 0

In [ ]:
train_data = train_data.loc[not0train_idx].reset_index(drop=True)
test_data = test_data.loc[not0test_idx].reset_index(drop=True)

In [ ]:
x_train = x_train[not0train_idx]
y_train = y_train[not0train_idx]

x_test = x_test[not0test_idx]
y_test = y_test[not0test_idx]

# 모델링

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPool2D
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, SimpleRNN, GRU

from tensorflow.keras.models import Sequential

In [ ]:
keras.backend.clear_session()

model = Sequential()

model.add(Embedding( max_words, embedding_dim, input_length=max_len ))

model.add(Conv1D(128, 5, activation='swish'))
forward_layer = LSTM(48, return_sequences=True )
backward_layer = GRU(48, return_sequences=True, go_backwards=True)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
model.add(Bidirectional(GRU(64, return_sequences=True)))
model.add(Flatten())
model.add(Dense(1024, activation='swish'))
model.add(Dense(1024, activation='swish'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer = keras.optimizers.Adam(),
              metrics = 'accuracy')

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1)

model.fit(x_train, y_train, epochs=100, batch_size=1024, # 일반적으로 이렇게 크게 주진 않는다.
          validation_split=0.2, verbose =1,
          callbacks=[es])

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
review_idx = 456

temp = test_data.loc[review_idx]
docu = temp['document']
label = 'positive' if temp['label'] ==1 else 'Negative'

print(f"문서 번호 {review_idx}")
print(label, " : ", docu)
y_pred = model.predict(x_test[review_idx:review_idx+1])
label_pred = 'positive' if y_pred[0,0] >=0.5 else 'Negative'
print(f"모델의 예측 : {label_pred},   prob = {y_pred[0,0]*100:.2f}%")